Conditional Elements: Composing Patterns Together

AND creates a composed pattern containing all Facts passed as arguments. All of the passed patterns must match for the composed pattern to match.


In [ ]:
from experta import *
#Match if two facts are declared, one matching Fact(1) and other matching Fact(2)
@Rule(AND(Fact(1),
          Fact(2)))
def _():
    pass

OR creates a composed pattern in which any of the given pattern will make the rule match.



In [ ]:
#Match if a fact matching Fact(1) exists and/or a fact matching Fact(2) exists
@Rule(OR(Fact(1),
         Fact(2)))
def _():
    pass

#If multiple facts match, the rule will be fired multiple times, one for each valid combination of matching facts.

NOT
This element matches if the given pattern does not match with any fact or combination of facts. Therefore this element matches the absence of the given pattern.



In [ ]:
@Rule(NOT(Fact(1)))
def _():
    pass

#Match if no fact match with Fact(1)

TEST
Check the received callable against the current binded values. If the execution returns True the evaluation will continue and stops otherwise.

In [ ]:
class Number(Fact):
    pass

@Rule(Number(MATCH.a),                   # Bind a variable 'a' to the value of a Number fact
        Number(MATCH.b),                   # Bind 'b' to another Number fact
        TEST(lambda a, b: a > b),          # Only continue if a > b
        Number(MATCH.c),                   # Bind 'c' to another Number fact
        TEST(lambda b, c: b > c)   )       # Only continue if b > c

def _(a, b, c):
    pass
#Match for all numbers a, b, c where a > b > c

EXISTS

This Conditional Elements receives a pattern and matches if one or more facts matches this pattern. This will match only once while one or more matching facts exists and will stop matching when there is no matching facts.

In [ ]:
class Color(Fact):
    pass
@Rule(EXISTS(Color()))
def _():
    pass

The FORALL conditional element provides a mechanism for determining if a group of specified CEs is satisfied for every occurence of another specified CE.



In [ ]:
#Match when for every Student fact there is a Reading, Writing and Arithmetic fact with the same name.
class Student(Fact):
    pass
class Reading(Fact):
    pass
class Writing(Fact):
    pass
class Arithmetic(Fact):
    pass
@Rule(FORALL(Student(MATCH.name),
             Reading(MATCH.name),
             Writing(MATCH.name),
             Arithmetic(MATCH.name)))
def all_students_passed():
    pass

In [ ]:
class Student(Fact):
    pass

class Laptop(Fact):
    pass

@Rule(FORALL(Student(MATCH.name),
             Laptop(MATCH.name)))
def all_students_passed():
    pass

#every student has a laptop

In [ ]:
class Laptop(Fact):
    """Laptop ownership fact with student name."""
    pass

class DeviceEngine(KnowledgeEngine):

    @Rule(
        FORALL(
            Student(MATCH.name),       # For every Student
            Laptop(MATCH.name)         # There must be a matching Laptop fact
        )
    )
    def all_have_laptops(self):
        print("✅ All students have laptops.")


Field Constraints: FC for sort

L (Literal Field Constraint)
This element performs an exact match with the given value. The matching is done using the equality operator ==.

In [ ]:
@Rule(Fact(L(3)))
def _():
    pass
#Match if the first element is exactly 3
#This is the default FC used when no FC is given as a pattern value. pattern.

W (Wildcard Field Constraint)

This element matches with any value.

In [ ]:
# Match if some fact is declared with the key mykey.
@Rule(Fact(mykey=W()))
def _():
    pass
# This element only matches if the element exist.

P (Predicate Field Constraint)

The match of this element is the result of applying the given callable to the fact-extracted value. If the callable returns True the FC will match, in other case the FC will not match.

In [ ]:
#Match if some fact is declared whose first parameter is an instance of int
@Rule(Fact(P(lambda x: isinstance(x, int))))
def _():
    pass

Composing FCs: &, | and ~
    All FC can be composed together using the composition operators &, | and ~.

ANDFC() a.k.a. &
The composed FC matches if all the given FC match.

In [ ]:
@Rule(Fact(x=P(lambda x: x >= 0) & P(lambda x: x <= 255)))
def _():
    pass
# Match if key x of Point is a value between 0 and 255.

ORFC() a.k.a. |
The composed FC matches if any of the given FC matches.

In [ ]:
@Rule(Fact(name=L('Alice') | L('Bob')))
def _():
    pass

NOTFC() a.k.a. ~
This composed FC negates the given FC, reversing the logic. If the given FC matches this will not and vice versa.



In [ ]:
@Rule(Fact(name=~L('Charlie')))
def _():
    pass
# Match if name is not Charlie.

MATCH object

The MATCH objects helps generating more readable name bindings. Is syntactic sugar for a Wildcard Field Constraint binded to a name. For example:

In [ ]:
@Rule(Fact(MATCH.myvalue))
def _(myvalue):
    pass

Is exactly the same as:

In [ ]:
@Rule(Fact("myvalue" << W()))
def _(myvalue):
    pass

AS object

The AS object like the MATCH object is syntactic sugar for generating bindable names. In this case any attribute requested to the AS object will return a string with the same name.

In [ ]:
@Rule(AS.myfact << Fact(W()))
def _(myfact):
    pass

Nested matching
Nested matching is useful to match against Fact values which contains nested structures like dicts or lists.

In [ ]:
Fact(name="scissors", against={"scissors": 0, "rock": -1, "paper": 1})
Fact(name="paper", against={"scissors": -1, "rock": 1, "paper": 0})
Fact(name="rock", against={"scissors": 1, "rock": 0, "paper": -1})

Fact(name='rock', against=<frozendict {'scissors': 1, 'rock': 0, 'paper': -1}>)

Nested matching take the form field__subkey=value. (That’s a double-underscore). For example:

In [ ]:
@Rule(Fact(name=MATCH.name, against__scissors=1, against__paper=-1))
def what_wins_to_scissors_and_losses_to_paper(self, name):
        print(name)

Is possible to match against an arbitrary deep structure following the same method.

In [ ]:
class Ship(Fact):
    pass

In [ ]:
Ship(data={
    "name": "SmallShip",
    "position": {
        "x": 300,
        "y": 200},
    "parent": {
        "name": "BigShip",
        "position": {
            "x": 150,
            "y": 300}}})

Ship(data=<frozendict {'name': 'SmallShip', 'position': <frozendict {'x': 300, 'y': 200}>, 'parent': <frozendict {'name': 'BigShip', 'position': <frozendict {'x': 150, 'y': 300}>}>}>)

In [ ]:
#In this example we can check for collision between a ship and its parent with the following rule:
@Rule(Ship(data__name=MATCH.name1,
            data__position__x=MATCH.x,
            data__position__y=MATCH.y,
            data__parent__name=MATCH.name2,
            data__parent__position__x=MATCH.x,
            data__parent__position__y=MATCH.y))
def collision_detected(self, name1, name2, **_):
    print("COLLISION!", name1, name2)

If the nested data structure contains list, tuples or any other sequence you can use numeric indexes as needed.

In [ ]:
Ship(data={
    "name": "SmallShip",
    "position": {
        "x": 300,
        "y": 200},
    "enemies": [
             {"name": "Destroyer"},
             {"name": "BigShip"}
    ]
})

@Rule(Ship(data__enemies__0__name="Destroyer"))
def next_enemy_is_destroyer(self):
        print("Bye byee!")

Mutable objects

Experta’s matching algorithm depends on the values of the declared facts being immutable

When a Fact is created, all its values are transformed to an immutable type if they are not. For this matter the method experta.utils.freeze is used internally.



In [ ]:
from experta.utils import unfreeze
class MutableTest(KnowledgeEngine):
    @Rule(Fact(v1=MATCH.v1, v2=MATCH.v2, v3=MATCH.v3))
    def is_immutable(self, v1, v2, v3):
        v2=unfreeze(v2)
        v2.append(5)
        print(type(v1), "is Immutable!")
        print(type(v2), "is mutable!")
        print(type(v3), "is Immutable!")

ke = MutableTest()
ke.reset()
ke.declare(Fact(v1={"a": 1, "b": 2}, v2=[1, 2, 3], v3={1, 2, 3}))
ke.run()

<class 'frozendict.frozendict'> is Immutable!
<class 'list'> is mutable!
<class 'frozenset'> is Immutable!


Example:
    
    we will learn how to modify Facts
    we will learn how to retract fact in second method # we can pass 2 options as asrgument to retract
                  1-id of fact
                  2-fact itself
    we will learn how use AS object to bind Fact for use in conclusions

In [ ]:
class Maximum(KnowledgeEngine):
    @Rule(NOT(Fact(max=W())))
    def init(self):
        self.declare(Fact(max=0))

    @Rule(Fact(val=MATCH.val),
          AS.m << Fact(max=MATCH.max),
          TEST(lambda max, val: val > max))
    def compute_max(self, m, val):
        self.modify(m, max=val)

    @Rule(AS.v << Fact(val=MATCH.val),
          Fact(max=MATCH.max),
          TEST(lambda max, val: val <= max))
    def remove_val(self, v):
        self.retract(v)

    @Rule(AS.v << Fact(max=W()),
          NOT(Fact(val=W())))
    def print_max(self, v):
        print("Max:", v['max'])

In [ ]:
m = Maximum()
m.reset()
m.declare(*[Fact(val=x) for x in (12, 33, 42, 99, 55, 11, 75)])
m.run()

Max: 99


In [ ]:
m.facts

FactList([(0, InitialFact()), (10, Fact(max=99))])

In [ ]:
class Minimum(KnowledgeEngine):
    @Rule(NOT(Fact(min=W())))
    def init(self):
        self.declare(Fact(min=100))

    @Rule(Fact(val=MATCH.val),
          AS.m << Fact(min=MATCH.min),
          TEST(lambda min, val: val < min))
    def compute_min(self, m, val):
        self.modify(m, min=val)

    @Rule(AS.v << Fact(val=MATCH.val),
          Fact(min=MATCH.min),
          TEST(lambda min, val: val >= min))
    def remove_val(self, v):
        self.retract(v)

    @Rule(AS.v << Fact(min=W()),
          NOT(Fact(val=W())))
    def print_min(self, v):
        print("Min:", v['min'])

In [ ]:
m = Minimum()
m.reset()
m.declare(*[Fact(val=x) for x in (12, 33, 42, 99, 55, 11, 75)])
m.run()

Min: 11


In [ ]:
m.facts

FactList([(0, InitialFact()), (10, Fact(min=11))])

In [ ]:
# Error when len(args) != no of actual arguments
# required by the function

# args = [0, 1, 4, 9]


# def func(a, b, c):
# 	return a + b + c


# # calling function with unpacking args
# func(*args)


In [ ]:
class neighbor(Fact):
    pass

class holds(Fact):
    pass

class OrderNumber(KnowledgeEngine):
    @DefFacts()
    def _initial_action(self):
        yield neighbor(left="Ann",right="Bert")
        yield neighbor(left="Bert",right="Carol")
        yield neighbor(left="Carol",right="Dany")
        yield holds(person="Ann",number=17)
        yield holds(person="Bert",number=6)
        yield holds(person="Carol",number=7)
        yield holds(person="Dany",number=27)


    @Rule(neighbor(left=MATCH.a,right=MATCH.b),
          AS.Ahas << holds(person=MATCH.a,number=MATCH.x),
          AS.Bhas <<holds(person=MATCH.b,number=MATCH.y),
          TEST(lambda x,y:y>x)
    )
    def swap(self,Ahas,Bhas,x,y):
        self.modify(Ahas, number=y)
        self.modify(Bhas, number=x)

In [ ]:
o_n=OrderNumber()
o_n.reset()
o_n.run()
o_n.facts

FactList([(0, InitialFact()),
          (1, neighbor(left='Ann', right='Bert')),
          (2, neighbor(left='Bert', right='Carol')),
          (3, neighbor(left='Carol', right='Dany')),
          (12, holds(person='Carol', number=7)),
          (13, holds(person='Dany', number=6)),
          (14, holds(person='Ann', number=27)),
          (15, holds(person='Bert', number=17))])

# Exercise


                                                                                    بناء نظام خبير يقوم ب   -

                                                                                   تمثيل شجرة العائلة

                                        تحقيق العلاقات العائلية (الاب - الام -الاخ -الاسلاف -الجد -الجدة  )

                 ايجاد محارم شخص (الام -الجدات -البنات -الاخوات -بنات الاخوات-بنات الاخوة-العمات -الخالات)